In [159]:
import pandas as pd
import json
from datetime import datetime
from dateutil import parser
import math
import numpy as np

## metadata.json

In [16]:
metadata = {}
for img_id in [f"{i:06d}" for i in range(100000)]:
    fp = f"infos/single_frames/{img_id}/metadata.json"
    with open(fp, "r") as file:
        metadata[img_id] = json.load(file)

In [93]:
# Format into dataframe first
keys = ["frame_id", "time", "country_code", "scraped_weather", "road_type", "road_condition", "time_of_day", "num_vehicles", "longitude", "latitude", "solar_angle_elevation"]
rows = []
for _, item in metadata.items():
    row = []
    for key in keys:
        row.append(item[key])
    rows.append(row)

metadata_df = pd.DataFrame(rows, columns=keys)
metadata_df.head()

,frame_id,time,country_code,scraped_weather,road_type,road_condition,time_of_day,num_vehicles,longitude,latitude,solar_angle_elevation
0,000000,2021-04-19T10:23:10.444124Z,PL,partly-cloudy-day,city,normal,day,36,20.993243,52.242052,49.172337
1,000001,2021-04-27T10:51:26.001497Z,PL,partly-cloudy-day,city,normal,day,22,21.043137,52.249511,51.723833
2,000002,2021-04-18T16:04:19.891679Z,PL,cloudy,city,normal,day,3,21.030446,52.239373,13.700387
3,000003,2021-04-18T16:01:18.888716Z,PL,cloudy,city,normal,day,1,21.017903,52.237139,14.164055
4,000004,2021-04-18T16:16:17.336920Z,PL,cloudy,city,normal,day,42,21.005466,52.228612,11.891281


0. Start a new dataframe to hold the converted data

In [95]:
metadata_converted_df = pd.DataFrame()
metadata_converted_df["image_id"] = metadata_df["frame_id"]

1. Convert time from ISO 8601 format to Unix time

In [96]:
def iso_to_unix(iso_str):
    dt = parser.isoparse(iso_str)  # Parse ISO 8601 format
    unix_timestamp = dt.timestamp()  # Convert to Unix time (float)
    return unix_timestamp

# Perform conversion on the "time" column
metadata_converted_df["time"] = metadata_df["time"].apply(iso_to_unix)

metadata_converted_df.head()

,image_id,time
0,000000,1.618828e+09
1,000001,1.619521e+09
2,000002,1.618762e+09
3,000003,1.618762e+09
4,000004,1.618763e+09


2. Convert country_code to numerical labels

In [97]:
metadata_converted_df["country_code"], encoding_cc = pd.factorize(metadata_df["country_code"])

metadata_converted_df.head()

,image_id,time,country_code
0,000000,1.618828e+09,0
1,000001,1.619521e+09,0
2,000002,1.618762e+09,0
3,000003,1.618762e+09,0
4,000004,1.618763e+09,0


In [98]:
print("Category Mapping:", dict(enumerate(encoding_cc)))

Category Mapping: {0: 'PL', 1: 'DE', 2: 'IT', 3: 'HU', 4: 'SE', 5: 'FR', 6: 'NO', 7: 'GB', 8: 'IE', 9: 'NL', 10: 'CZ', 11: 'FI', 12: 'SK', 13: 'LU'}


3. Convert scraped_weather to numerical value

In [99]:
print(metadata_df["scraped_weather"].unique())

['partly-cloudy-day' 'cloudy' 'clear-day' 'rain' 'clear-night' 'snow'
 'partly-cloudy-night' 'wind' 'fog']


In [100]:
'''
A mannual encoding is performed here, with the following rationale:
- 0 = Clear-day: Ideal lighting, no obstructions.
- 1 = Clear-night: Less ideal than daytime due to reduced lighting, but still clear.
- 2 = Partly-cloudy-day: Slightly reduced contrast, but mostly clear.
- 3 = Partly-cloudy-night: Similar to clear-night but with clouds reducing moonlight.
- 4 = Cloudy: Diffused lighting, lower contrast but no major obstructions.
- 5 = Wind: Motion blur can be an issue, especially for lightweight objects.
- 6 = Rain: Water droplets on the lens, reflections, and reduced visibility.
- 7 = Snow: More occlusion than rain, with objects blending into the white background.
- 8 = Fog: The most challenging: heavy occlusion, low contrast, and objects may be completely invisible.
'''
def weather_encoding(weather):
    encoding = {
        "clear-day": 0,
        "clear-night": 1,
        "partly-cloudy-day": 2,
        "partly-cloudy-night": 3,
        "cloudy": 4,
        "wind": 5,
        "rain": 6,
        "snow": 7,
        "fog": 8 
    }

    return encoding[weather]

metadata_converted_df["weather"] = metadata_df["scraped_weather"].apply(weather_encoding)

metadata_converted_df.head()

,image_id,time,country_code,weather
0,000000,1.618828e+09,0,2
1,000001,1.619521e+09,0,2
2,000002,1.618762e+09,0,4
3,000003,1.618762e+09,0,4
4,000004,1.618763e+09,0,4


In [101]:
encoding_weather = {
        0: "clear-day",
        1: "clear-night",
        2: "partly-cloudy-day",
        3: "partly-cloudy-night",
        4: "cloudy",
        5: "wind",
        6: "rain",
        7: "snow",
        8: "fog" 
}

3. Convert road type and road condition

In [102]:
metadata_converted_df["road_type"], encoding_rt = pd.factorize(metadata_df["road_type"])
metadata_converted_df["road_condition"], encoding_rc = pd.factorize(metadata_df["road_condition"])

metadata_converted_df.head()

,image_id,time,country_code,weather,road_type,road_condition
0,000000,1.618828e+09,0,2,0,0
1,000001,1.619521e+09,0,2,0,0
2,000002,1.618762e+09,0,4,0,0
3,000003,1.618762e+09,0,4,0,0
4,000004,1.618763e+09,0,4,0,0


In [103]:
print("Category Mapping:", dict(enumerate(encoding_rt)))
print("Category Mapping:", dict(enumerate(encoding_rc)))

Category Mapping: {0: 'city', 1: 'arterial-urban', 2: 'highway', 3: 'arterial-rural', 4: 'smaller-rural'}
Category Mapping: {0: 'normal', 1: 'wet', 2: 'snow'}


4. Convert time of day

In [104]:
metadata_converted_df["time_of_day"], encoding_td = pd.factorize(metadata_df["time_of_day"])

metadata_converted_df.head()

,image_id,time,country_code,weather,road_type,road_condition,time_of_day
0,000000,1.618828e+09,0,2,0,0,0
1,000001,1.619521e+09,0,2,0,0,0
2,000002,1.618762e+09,0,4,0,0,0
3,000003,1.618762e+09,0,4,0,0,0
4,000004,1.618763e+09,0,4,0,0,0


In [105]:
print("Category Mapping:", dict(enumerate(encoding_td)))

Category Mapping: {0: 'day', 1: 'twilight', 2: 'night'}


5. Copy the rest, which does not need conversion

In [106]:
metadata_converted_df["num_vehicles"] = metadata_df["num_vehicles"]
metadata_converted_df["longitude"] = metadata_df["longitude"]
metadata_converted_df["latitude"] = metadata_df["latitude"]
metadata_converted_df["solar_angle_elevation"] = metadata_df["solar_angle_elevation"]

metadata_converted_df.head()

,image_id,time,country_code,weather,road_type,road_condition,time_of_day,num_vehicles,longitude,latitude,solar_angle_elevation
0,000000,1.618828e+09,0,2,0,0,0,36,20.993243,52.242052,49.172337
1,000001,1.619521e+09,0,2,0,0,0,22,21.043137,52.249511,51.723833
2,000002,1.618762e+09,0,4,0,0,0,3,21.030446,52.239373,13.700387
3,000003,1.618762e+09,0,4,0,0,0,1,21.017903,52.237139,14.164055
4,000004,1.618763e+09,0,4,0,0,0,42,21.005466,52.228612,11.891281


6. Record the encoding schema

In [107]:
metadata_encoding_schema = {
    "country_code": dict(enumerate(encoding_cc)),
    "weather": encoding_weather,
    "time_of_day": dict(enumerate(encoding_td)),
    "road_type": dict(enumerate(encoding_rt)),
    "road_condition": dict(enumerate(encoding_rc))
}

# same as a json file
with open("outputs/metadata_encoding.json", "w") as file:
    json.dump(metadata_encoding_schema, file, indent=4)

7. Save the converted metadata to the corresponding folder

In [110]:
stops = [5001, 10001, 15001, 20001, 25001, 30001, 35001, 40001, 45001, 50001]

for stop in stops:
    frame_id_int = (
        metadata_converted_df["image_id"]
        .astype(str)
        .str.lstrip("0")
        .replace("", "0")
        .astype(int)
    )
    selector = (frame_id_int >= stop - 5000) & (frame_id_int < stop)

    metadata_converted_df[selector].to_csv(
        f"outputs/{stop-5000}_{stop-1}/metadata/metadata_{stop-5000}_{stop-1}.csv"
    )

## calibration.json

In [90]:
calibration = {}
for img_id in [f"{i:06d}" for i in range(50000)]:
    fp = f"infos/single_frames/{img_id}/calibration.json"
    with open(fp, "r") as file:
        calibration[img_id] = json.load(file)

In [121]:
rows = []
for img_id, calibration_img in calibration.items():
    calibration_img = pd.json_normalize(calibration_img, sep='_')
    calibration_img.insert(0, "image_id", img_id)
    rows.append(calibration_img)

calibration_df = pd.concat(rows, ignore_index=True, sort=False)

In [128]:
calibration_df.head()

,image_id,FC_camera_type,FC_intrinsics,FC_extrinsics,FC_lidar_extrinsics,FC_image_dimensions,FC_distortion,FC_field_of_view,FC_xi,FC_undistortion
0,000000,kannala,"[[1866.254649589019, 0.0, 1919.254029415608, 0...","[[0.0031491181849355957, 0.007393642085207038,...","[[-0.03245669160011588, 0.9994709339337041, 0....","[3848, 2168]","[-0.02135201813326693, 0.01742444071089098, -0...","[120.0310462140434, 66.95362481129119]",None,"[0.02152830737867924, -0.01753514154278558, 0...."
1,000001,kannala,"[[1859.746969636308, 0.0, 1914.204755444865, 0...","[[0.007860833774959095, 0.02827074644968561, 0...","[[-0.03340051269954744, 0.9994218673478981, 0....","[3848, 2168]","[-0.02282673566953882, 0.01933894662682144, -0...","[120.581700743319, 67.21490441580156]",None,"[0.02304399728590842, -0.01958384906699419, 0...."
2,000002,kannala,"[[1866.254649589019, 0.0, 1919.254029415608, 0...","[[0.0031491181849355957, 0.007393642085207038,...","[[-0.03245669160011588, 0.9994709339337041, 0....","[3848, 2168]","[-0.02135201813326693, 0.01742444071089098, -0...","[120.0310462140434, 66.95362481129119]",None,"[0.02152830737867924, -0.01753514154278558, 0...."
3,000003,kannala,"[[1866.254649589019, 0.0, 1919.254029415608, 0...","[[0.0031491181849355957, 0.007393642085207038,...","[[-0.03245669160011588, 0.9994709339337041, 0....","[3848, 2168]","[-0.02135201813326693, 0.01742444071089098, -0...","[120.0310462140434, 66.95362481129119]",None,"[0.02152830737867924, -0.01753514154278558, 0...."
4,000004,kannala,"[[1866.254649589019, 0.0, 1919.254029415608, 0...","[[0.0031491181849355957, 0.007393642085207038,...","[[-0.03245669160011588, 0.9994709339337041, 0....","[3848, 2168]","[-0.02135201813326693, 0.01742444071089098, -0...","[120.0310462140434, 66.95362481129119]",None,"[0.02152830737867924, -0.01753514154278558, 0...."


0. Prepare a new dataframe for the converted data

In [126]:

calibration_converted_df = pd.DataFrame()
calibration_converted_df["image_id"] = calibration_df["image_id"]

calibration_converted_df.head()

,image_id
0,000000
1,000001
2,000002
3,000003
4,000004


1. Extract features from FC_intrinsics

In [133]:
"""
Explanation of camera intrinsics: https://ksimek.github.io/2013/08/13/intrinsic/

The format of intrinsics:
First row: fx, 0, cx, 0 (focal length in x, principal point x)
Second row: 0, fy, cy, 0 (focal length in y, principal point y)
Third row: 0, 0, 1, 0 (homogeneous coordinates)

"""


def extract_focal_length_x(matrix):
    return matrix[0][0]


def extract_focal_length_y(matrix):
    return matrix[1][1]


def extract_principle_point_x(matrix):
    return matrix[0][2]


def extract_principle_point_y(matrix):
    return matrix[0][2]


calibration_converted_df["focal_length_x"] = calibration_df["FC_intrinsics"].apply(
    extract_focal_length_x
)
calibration_converted_df["focal_length_y"] = calibration_df["FC_intrinsics"].apply(
    extract_focal_length_y
)
calibration_converted_df["principle_point_x"] = calibration_df["FC_intrinsics"].apply(
    extract_principle_point_x
)
calibration_converted_df["principle_point_y"] = calibration_df["FC_intrinsics"].apply(
    extract_principle_point_x
)

calibration_converted_df.head()

,image_id,focal_length_x,focal_length_y,principle_point_x
0,000000,1866.25465,1866.25465,1919.254029
1,000001,1859.74697,1859.74697,1914.204755
2,000002,1866.25465,1866.25465,1919.254029
3,000003,1866.25465,1866.25465,1919.254029
4,000004,1866.25465,1866.25465,1919.254029


2. Extract features from FC_extrinsics

In [ ]:
'''
Raw data:
The first 3×3 part represents the rotation matrix.
The last column (first three rows) represents the translation vector (x, y, z) in meters.

Conversion:
1. Convert the rotation matrix to Euler angles: yaw, pitch, roll
2. The translation vector directly gives the position of the camera in the world coordinate system.
'''

def extract_pose_x(matrix):
    return matrix[0][-1]

def extract_pose_y(matrix):
    return matrix[1][-1]

def extract_pose_z(matrix):
    return matrix[2][-1]

def extract_pose_yaw(matrix):
    yaw = math.atan2(matrix[1][0], matrix[0][0])
    return yaw

def extract_pose_pitch(matrix):
    pitch = math.asin(-matrix[2][0])
    return pitch

def extract_pose_roll(matrix):
    roll = math.atan2(matrix[2][1], matrix[2][2])
    return roll

calibration_converted_df["pose_x"] = calibration_df["FC_extrinsics"].apply(extract_pose_x)
calibration_converted_df["pose_y"] = calibration_df["FC_extrinsics"].apply(extract_pose_y)
calibration_converted_df["pose_yaw"] = calibration_df["FC_extrinsics"].apply(extract_pose_yaw)
calibration_converted_df["pose_pitch"] = calibration_df["FC_extrinsics"].apply(extract_pose_pitch)
calibration_converted_df["pose_roll"] =  calibration_df["FC_extrinsics"].apply(extract_pose_roll)

calibration_converted_df.head()

,image_id,focal_length_x,focal_length_y,principle_point_x,pose_x,pose_y,pose_yaw,pose_pitch,pose_roll
0,000000,1866.25465,1866.25465,1919.254029,2.023074,1.129931,-1.567647,0.013673,-1.563360
1,000001,1859.74697,1859.74697,1914.204755,2.009231,1.136123,-1.562932,0.029530,-1.542289
2,000002,1866.25465,1866.25465,1919.254029,2.023074,1.129931,-1.567647,0.013673,-1.563360
3,000003,1866.25465,1866.25465,1919.254029,2.023074,1.129931,-1.567647,0.013673,-1.563360
4,000004,1866.25465,1866.25465,1919.254029,2.023074,1.129931,-1.567647,0.013673,-1.563360


3. Extract features from FC_field_of_view

In [140]:
'''
The original format is: horizontal FOV, vertical FOV
Simply flatten the data.
'''
calibration_converted_df["horizontal_fov"] = calibration_df["FC_field_of_view"].apply(lambda x: x[0])
calibration_converted_df["vertical_fov"] = calibration_df["FC_field_of_view"].apply(lambda x: x[1])

calibration_converted_df.head()

,image_id,focal_length_x,focal_length_y,principle_point_x,pose_x,pose_y,pose_yaw,pose_pitch,pose_roll,horizontal_fov,vertical_fov
0,000000,1866.25465,1866.25465,1919.254029,2.023074,1.129931,-1.567647,0.013673,-1.563360,120.031046,66.953625
1,000001,1859.74697,1859.74697,1914.204755,2.009231,1.136123,-1.562932,0.029530,-1.542289,120.581701,67.214904
2,000002,1866.25465,1866.25465,1919.254029,2.023074,1.129931,-1.567647,0.013673,-1.563360,120.031046,66.953625
3,000003,1866.25465,1866.25465,1919.254029,2.023074,1.129931,-1.567647,0.013673,-1.563360,120.031046,66.953625
4,000004,1866.25465,1866.25465,1919.254029,2.023074,1.129931,-1.567647,0.013673,-1.563360,120.031046,66.953625


4. Save the converted calibrations to the corresponding folder.

In [141]:
stops = [5001, 10001, 15001, 20001, 25001, 30001, 35001, 40001, 45001, 50001]

for stop in stops:
    frame_id_int = (
        calibration_converted_df["image_id"]
        .astype(str)
        .str.lstrip("0")
        .replace("", "0")
        .astype(int)
    )
    selector = (frame_id_int >= stop - 5000) & (frame_id_int < stop)

    calibration_converted_df[selector].to_csv(
        f"outputs/{stop-5000}_{stop-1}/metadata/calibration_{stop-5000}_{stop-1}.csv"
    )

## ego_motion.json

In [142]:
ego_motion = {}
for img_id in [f"{i:06d}" for i in range(50000)]:
    fp = f"infos/single_frames/{img_id}/ego_motion.json"
    with open(fp, "r") as file:
        ego_motion[img_id] = json.load(file)

In [154]:
keys = ["timestamps", "poses", "velocities", "accelerations", "angular_rates"]
rows = []
for image_id, item in ego_motion.items():
    row = [image_id]
    for key in keys:
        row.append(item[key])
    rows.append(row)
    
ego_motion_df = pd.DataFrame(rows, columns=["image_id"]+keys)
ego_motion_df.head()

,image_id,timestamps,poses,velocities,accelerations,angular_rates
0,000000,"[1618827789.306996, 1618827789.417964, 1618827...","[[[0.9999599331042809, -0.005123249974757692, ...","[[8.150486284637038, 0.050889431248160005, 0.0...","[[0.2642775433326181, -0.2927040268936414, -8....","[[-0.22109078416851724, 1.0224266904883779, -0..."
1,000001,"[1619520684.862908, 1619520684.973858, 1619520...","[[[0.9999481343908736, 0.010180078920862702, -...","[[9.836411477001484, -0.16136038807937653, 0.0...","[[-0.008201478196600148, 0.018366094461531125,...","[[-0.3804590160577624, -0.25942025774057587, 0..."
2,000002,"[1618761858.751402, 1618761858.862374, 1618761...","[[[0.9999997699360017, -0.0006698817173896102,...","[[-0.0008900282297242693, 0.001024191030791775...","[[0.2102523782275946, -0.20420841856805666, -9...","[[0.033022466167315426, -0.04248969112149266, ..."
3,000003,"[1618761677.751436, 1618761677.862407, 1618761...","[[[0.9997733098192796, -0.02095781770901292, 0...","[[2.4503526673958578, 0.013190865206345489, -0...","[[-0.5083811553775363, -0.00899844074100565, -...","[[0.06766560116648171, 0.243674593970332, -0.6..."
4,000004,"[1618762576.195988, 1618762576.306959, 1618762...","[[[0.9999094929585226, 0.01335604258760522, 0....","[[12.657233747711123, 0.023285435444421385, 0....","[[0.007874412893689054, -0.047534675631213795,...","[[-0.041167515501415, -0.3040280608455702, 0.1..."


1. Prepare a new dataframe for the converted data

In [165]:
ego_motion_converted_df = pd.DataFrame()
ego_motion_converted_df["image_id"] = ego_motion_df["image_id"]

ego_motion_converted_df.head()

,image_id
0,000000
1,000001
2,000002
3,000003
4,000004


2. Select the pose at the middle timestamp, and convert to x, y, z, yaw, pitch, and roll

In [166]:
def extract_pose_x(matrices):
    middle_idx = int(len(matrices) / 2)
    matrix = matrices[middle_idx]
    return matrix[0][-1]

def extract_pose_y(matrices):
    middle_idx = int(len(matrices) / 2)
    matrix = matrices[middle_idx]
    return matrix[1][-1]

def extract_pose_y(matrices):
    middle_idx = int(len(matrices) / 2)
    matrix = matrices[middle_idx]
    return matrix[2][-1]

def extract_pose_yaw(matrices):
    middle_idx = int(len(matrices) / 2)
    matrix = matrices[middle_idx]
    yaw = math.atan2(matrix[1][0], matrix[0][0])
    return yaw

def extract_pose_pitch(matrices):
    middle_idx = int(len(matrices) / 2)
    matrix = matrices[middle_idx]
    pitch = math.asin(-matrix[2][0])
    return pitch

def extract_pose_roll(matrices):
    middle_idx = int(len(matrices) / 2)
    matrix = matrices[middle_idx]
    roll = math.atan2(matrix[2][1], matrix[2][2])
    return roll

ego_motion_converted_df["pose_x"] = ego_motion_df["poses"].apply(extract_pose_x)
ego_motion_converted_df["pose_y"] = ego_motion_df["poses"].apply(extract_pose_y)
ego_motion_converted_df["pose_yaw"] = ego_motion_df["poses"].apply(extract_pose_yaw)
ego_motion_converted_df["pose_pitch"] = ego_motion_df["poses"].apply(extract_pose_pitch)
ego_motion_converted_df["pose_roll"] = ego_motion_df["poses"].apply(extract_pose_roll)

ego_motion_converted_df.head()

,image_id,pose_x,pose_y,pose_yaw,pose_pitch,pose_roll
0,000000,38.758484,-0.291283,0.005113,-0.001916,-0.001982
1,000001,49.135673,-0.250412,-0.017753,-0.004324,-0.010006
2,000002,0.004744,-0.031427,0.000925,0.000107,0.000232
3,000003,12.286499,-0.022671,0.031859,0.004122,0.003537
4,000004,63.348320,-0.045824,-0.008929,-0.003168,-0.011567


3. Derive speed variance

In [167]:
def compute_speed_variance(velocities):
    speeds = np.linalg.norm(velocities, axis=1)  # Compute speed magnitude
    variance = np.var(speeds)  # Compute variance
    return variance

ego_motion_converted_df["speed_var"] = ego_motion_df["velocities"].apply(compute_speed_variance)

ego_motion_converted_df.head()

,image_id,pose_x,pose_y,pose_yaw,pose_pitch,pose_roll,speed_var
0,000000,38.758484,-0.291283,0.005113,-0.001916,-0.001982,1.463442e-02
1,000001,49.135673,-0.250412,-0.017753,-0.004324,-0.010006,1.443066e-01
2,000002,0.004744,-0.031427,0.000925,0.000107,0.000232,1.066701e-07
3,000003,12.286499,-0.022671,0.031859,0.004122,0.003537,1.438465e-01
4,000004,63.348320,-0.045824,-0.008929,-0.003168,-0.011567,2.577095e-03


4. Compute jerk (rate of acceleration change)

In [169]:
def compute_jerk(accelerations, timestamps):
    accelerations = np.array(accelerations)
    timestamps = np.array(timestamps)

    dt = np.diff(timestamps)  # Time differences
    da = np.diff(accelerations, axis=0)  # Acceleration differences

    jerk = da / dt[:, np.newaxis]  # Compute jerk
    return np.mean(jerk), np.max(jerk), np.std(jerk)


jerk_ls = []

for _, row in ego_motion_df.iterrows():
    jerk = compute_jerk(row["accelerations"], row["timestamps"])
    jerk_ls.append(jerk)


ego_motion_converted_df[["mean_jerk", "max_jerk", "st_jerk"]] = jerk_ls

ego_motion_converted_df.head()

,image_id,pose_x,pose_y,pose_yaw,pose_pitch,pose_roll,speed_var,mean_jerk,max_jerk,st_jerk
0,000000,38.758484,-0.291283,0.005113,-0.001916,-0.001982,1.463442e-02,-0.052666,12.104334,3.098969
1,000001,49.135673,-0.250412,-0.017753,-0.004324,-0.010006,1.443066e-01,-0.018623,13.925508,5.708523
2,000002,0.004744,-0.031427,0.000925,0.000107,0.000232,1.066701e-07,0.000140,0.330015,0.090981
3,000003,12.286499,-0.022671,0.031859,0.004122,0.003537,1.438465e-01,0.119149,2.305955,0.723185
4,000004,63.348320,-0.045824,-0.008929,-0.003168,-0.011567,2.577095e-03,0.092260,7.090513,2.489161


5. Derive angular acceleration

In [170]:
def compute_angular_acceleration(angular_rates, timestamps):
    angular_rates = np.array(angular_rates)
    timestamps = np.array(timestamps)

    dt = np.diff(timestamps)  # Time differences
    dw = np.diff(angular_rates, axis=0)  # Angular rate differences

    angular_acceleration = dw / dt[:, np.newaxis]
    return (
        np.mean(angular_acceleration),
        np.max(angular_acceleration),
        np.std(angular_acceleration),
    )


angular_acc_ls = []

for _, row in ego_motion_df.iterrows():
    angular_acc = compute_angular_acceleration(row["angular_rates"], row["timestamps"])
    angular_acc_ls.append(angular_acc)


ego_motion_converted_df[["mean_angular_acc", "max_angular_acc", "st_angular_acc"]] = (
    angular_acc_ls
)

ego_motion_converted_df.head()

,image_id,pose_x,pose_y,pose_yaw,pose_pitch,pose_roll,speed_var,mean_jerk,max_jerk,st_jerk,mean_angular_acc,max_angular_acc,st_angular_acc
0,000000,38.758484,-0.291283,0.005113,-0.001916,-0.001982,1.463442e-02,-0.052666,12.104334,3.098969,0.159189,30.289053,8.191551
1,000001,49.135673,-0.250412,-0.017753,-0.004324,-0.010006,1.443066e-01,-0.018623,13.925508,5.708523,0.592639,36.686026,15.379224
2,000002,0.004744,-0.031427,0.000925,0.000107,0.000232,1.066701e-07,0.000140,0.330015,0.090981,0.018831,1.185745,0.520336
3,000003,12.286499,-0.022671,0.031859,0.004122,0.003537,1.438465e-01,0.119149,2.305955,0.723185,-0.177618,3.927114,1.717938
4,000004,63.348320,-0.045824,-0.008929,-0.003168,-0.011567,2.577095e-03,0.092260,7.090513,2.489161,-0.418295,20.698851,7.173318


5. Compute lateral acceleration

In [171]:
def compute_lateral_acceleration(velocities, angular_rates):
    speeds = np.linalg.norm(velocities, axis=1)  # Compute speed magnitude
    yaw_rates = np.array(angular_rates)[:, 2]  # Extract yaw rate (z-axis rotation)

    lateral_acceleration = speeds * yaw_rates
    return (
        np.mean(lateral_acceleration),
        np.max(lateral_acceleration),
        np.std(lateral_acceleration),
    )

lateral_acc_ls = []

for _, row in ego_motion_df.iterrows():
    lateral_acc = compute_lateral_acceleration(row["velocities"], row["angular_rates"])
    lateral_acc_ls.append(lateral_acc)


ego_motion_converted_df[["mean_lateral_acc", "max_lateral_acc", "st_lateral_acc"]] = (
    lateral_acc_ls
)

ego_motion_converted_df.head()

,image_id,pose_x,pose_y,pose_yaw,pose_pitch,pose_roll,speed_var,mean_jerk,max_jerk,st_jerk,mean_angular_acc,max_angular_acc,st_angular_acc,mean_lateral_acc,max_lateral_acc,st_lateral_acc
0,000000,38.758484,-0.291283,0.005113,-0.001916,-0.001982,1.463442e-02,-0.052666,12.104334,3.098969,0.159189,30.289053,8.191551,1.200247,7.135908,2.158875
1,000001,49.135673,-0.250412,-0.017753,-0.004324,-0.010006,1.443066e-01,-0.018623,13.925508,5.708523,0.592639,36.686026,15.379224,4.980040,10.819544,3.036366
2,000002,0.004744,-0.031427,0.000925,0.000107,0.000232,1.066701e-07,0.000140,0.330015,0.090981,0.018831,1.185745,0.520336,-0.000026,0.000212,0.000093
3,000003,12.286499,-0.022671,0.031859,0.004122,0.003537,1.438465e-01,0.119149,2.305955,0.723185,-0.177618,3.927114,1.717938,-1.132756,-0.942511,0.152678
4,000004,63.348320,-0.045824,-0.008929,-0.003168,-0.011567,2.577095e-03,0.092260,7.090513,2.489161,-0.418295,20.698851,7.173318,-1.517754,2.063336,1.839520


6. Derive a unified stability score

In [177]:
'''
Anything reduces stability discounts the score, which is defined as:

stability_score = 100 - (jerk variance + angular acceleration variance + lateral acceleration variance)

'''

def compute_stability_score(st_jerk, st_angular_acc, st_lateral_acc):
    jerk_variance = st_jerk ** 2
    angular_variance = st_angular_acc ** 2
    lateral_variance = st_lateral_acc ** 2

    stability_score = 100 - (jerk_variance + angular_variance + lateral_variance)
    return stability_score

score_ls = []

for _, row in ego_motion_converted_df.iterrows():
    score = compute_stability_score(row["st_jerk"], row["st_angular_acc"], row["st_lateral_acc"])
    score_ls.append(score)

ego_motion_converted_df["stability_score"] = score_ls

ego_motion_converted_df.head()

,image_id,pose_x,pose_y,pose_yaw,pose_pitch,pose_roll,speed_var,mean_jerk,max_jerk,st_jerk,mean_angular_acc,max_angular_acc,st_angular_acc,mean_lateral_acc,max_lateral_acc,st_lateral_acc,stability_score
0,000000,38.758484,-0.291283,0.005113,-0.001916,-0.001982,1.463442e-02,-0.052666,12.104334,3.098969,0.159189,30.289053,8.191551,1.200247,7.135908,2.158875,18.634144
1,000001,49.135673,-0.250412,-0.017753,-0.004324,-0.010006,1.443066e-01,-0.018623,13.925508,5.708523,0.592639,36.686026,15.379224,4.980040,10.819544,3.036366,-178.327288
2,000002,0.004744,-0.031427,0.000925,0.000107,0.000232,1.066701e-07,0.000140,0.330015,0.090981,0.018831,1.185745,0.520336,-0.000026,0.000212,0.000093,99.720973
3,000003,12.286499,-0.022671,0.031859,0.004122,0.003537,1.438465e-01,0.119149,2.305955,0.723185,-0.177618,3.927114,1.717938,-1.132756,-0.942511,0.152678,96.502382
4,000004,63.348320,-0.045824,-0.008929,-0.003168,-0.011567,2.577095e-03,0.092260,7.090513,2.489161,-0.418295,20.698851,7.173318,-1.517754,2.063336,1.839520,38.963753


7. Save the converted ego motion data to the corresponding folder

In [178]:
stops = [5001, 10001, 15001, 20001, 25001, 30001, 35001, 40001, 45001, 50001]

for stop in stops:
    frame_id_int = (
        ego_motion_converted_df["image_id"]
        .astype(str)
        .str.lstrip("0")
        .replace("", "0")
        .astype(int)
    )
    selector = (frame_id_int >= stop - 5000) & (frame_id_int < stop)

    ego_motion_converted_df[selector].to_csv(
        f"outputs/{stop-5000}_{stop-1}/metadata/ego_motion_{stop-5000}_{stop-1}.csv"
    )